# Notebook for preprocessing Wikipedia (Indonesia) dataset

### Initilizing phonemizer and tokenizer

In [1]:
import yaml

config_path = "Configs/config.yml" # you can change it to anything else
config = yaml.safe_load(open(config_path))

In [2]:
import sys
sys.path.insert(0, '/workspace/src/PL-BERT-ID')
from phonemize import phonemize, EnIndPhonemizer

In [3]:
global_phonemizer = EnIndPhonemizer(ipa=True, keep_stress=True, sep="")

In [4]:
import os
os.environ['TRUST_REMOTE_CODE'] = 'True'

In [5]:
from transformers import TransfoXLTokenizer
tokenizer = TransfoXLTokenizer.from_pretrained(config['dataset_params']['tokenizer']) # you can use any other tokenizers if you want to

/workspace/src/PL-BERT-ID/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
`TransfoXL` was deprecated due to security issues linked to `pickle.load` in `TransfoXLTokenizer`. See more details on this model's documentation page: `https://github.com/huggingface/transformers/blob/main/docs/source/en/model_doc/transfo-xl.md`.


### Process dataset

Since using load_dataset with the Indonesian Wikipedia (id) resulted in errors (e.g., "Not Found"), we will download and load the dataset manually.

You can download the dataset from this link: https://huggingface.co/datasets/wikimedia/wikipedia/tree/main/20231101.id.

In [6]:
from datasets import load_dataset

# Use a glob pattern to load all Parquet files in the 'wikipedia' folder.
# This pattern will search for all files ending with '.parquet' within the folder.
parquet_folder = "/workspace/src/PL-BERT-ID/wikipedia/*.parquet"

try:
    dataset = load_dataset("parquet", data_files=parquet_folder)
    if isinstance(dataset, dict) or hasattr(dataset, "keys"):
        split_name = "train" if "train" in dataset else list(dataset.keys())[0]
        dataset = dataset[split_name]
    print("Dataset loaded successfully!")
    print(dataset)
except Exception as e:
    print(f"An error occurred while loading the dataset: {e}")

Dataset loaded successfully!
Dataset({
    features: ['id', 'url', 'title', 'text'],
    num_rows: 665622
})


In [7]:
root_directory = "./wiki_phoneme" # set up root directory for multiprocessor processing

In [8]:
# Update the process_shard function with better error handling

import os
import time

num_shards = 100

def process_shard(i):
    directory = root_directory + "/shard_" + str(i)
    if os.path.exists(directory):
        print("Shard %d already exists!" % i)
        return f"Shard {i} already exists"
    
    start_time = time.time()
    print('Processing shard %d ...' % i)
    
    try:
        shard = dataset.shard(num_shards=num_shards, index=i)
        print(f'Shard {i}: {len(shard)} examples to process')
        
        # Process in smaller batches to avoid memory issues
        processed_dataset = shard.map(
            lambda t: phonemize(t['text'], global_phonemizer, tokenizer), 
            remove_columns=['text'],
            batch_size=100,  # Process in smaller batches
            desc=f"Phonemizing shard {i}"
        )
        
        if not os.path.exists(directory):
            os.makedirs(directory)
        processed_dataset.save_to_disk(directory)
        
        elapsed = time.time() - start_time
        print(f'Shard {i} completed in {elapsed:.2f} seconds')
        return f"Shard {i} completed successfully"
        
    except Exception as e:
        print(f'Shard {i} failed: {str(e)}')
        return Exception(f"Shard {i} failed: {str(e)}")

In [9]:
from pebble import ProcessPool
from concurrent.futures import TimeoutError

#### Note: You will need to run the following cell multiple times to process all shards because some will fail. Depending on how fast you process each shard, you will need to change the timeout to a longer value to make more shards processed before being killed.


In [10]:
# Update the processing cell with better timeout and worker settings

import os
from pebble import ProcessPool
from concurrent.futures import TimeoutError

# Reduce workers to avoid resource contention and increase timeout
max_workers = 8  # Reduced from 20
timeout_seconds = 1800  # Increased to 30 minutes
failed_shards = []

print(f"Processing {num_shards} shards with {max_workers} workers...")
print(f"Timeout set to {timeout_seconds} seconds ({timeout_seconds//60} minutes)")

with ProcessPool(max_workers=max_workers) as pool:
    future = pool.map(process_shard, range(num_shards), timeout=timeout_seconds)
    
    try:
        for i, result in enumerate(future.result()):
            if isinstance(result, Exception):
                print(f"Shard {i} failed: {result}")
                failed_shards.append(i)
            else:
                if i % 10 == 0:  # Progress indicator
                    print(f"Completed {i+1}/{num_shards} shards")
    except Exception as e:
        print(f"Processing error: {e}")

print(f"Processing completed. Failed shards: {len(failed_shards)}")
if failed_shards:
    print(f"Failed shard indices: {failed_shards}")

Processing 100 shards with 8 workers...
Timeout set to 1800 seconds (30 minutes)
Processing shard 0 ...Processing shard 1 ...Processing shard 2 ...Processing shard 3 ...



Processing shard 4 ...Processing shard 5 ...

Processing shard 7 ...Processing shard 6 ...

Shard 0: 6657 examples to processShard 1: 6657 examples to process
Shard 2: 6657 examples to processShard 3: 6657 examples to process
Shard 4: 6657 examples to process

Shard 5: 6657 examples to process

Shard 6: 6657 examples to processShard 7: 6657 examples to process



Phonemizing shard 7:  48%|████▊     | 3178/6657 [10:13<01:57, 29.52 examples/s]

Shard 5 completed in 613.96 seconds
Processing shard 8 ...
Shard 8: 6657 examples to process


Phonemizing shard 0:  21%|██        | 1404/6657 [13:40<57:10,  1.53 examples/s]  

Shard 7 completed in 820.88 seconds
Processing shard 9 ...
Shard 9: 6657 examples to process


...

Shard 98 completed in 464.74 seconds


Saving the dataset (1/1 shards): 100%|██████████| 6656/6656 [00:00<00:00, 72810.49 examples/s]


Shard 96 completed in 538.35 seconds


Saving the dataset (1/1 shards): 100%|██████████| 6656/6656 [00:00<00:00, 77864.02 examples/s]


Shard 99 completed in 379.85 seconds
Processing completed. Failed shards: 0


In [11]:
# Add this cell before processing to check existing shards

def check_existing_shards():
    """Check which shards already exist"""
    existing_shards = []
    pending_shards = []
    
    for i in range(num_shards):
        directory = root_directory + "/shard_" + str(i)
        if os.path.exists(directory):
            existing_shards.append(i)
        else:
            pending_shards.append(i)
    
    print(f"Existing shards: {len(existing_shards)}/{num_shards}")
    print(f"Pending shards: {len(pending_shards)}")
    
    return pending_shards

# Check existing shards first
pending_shards = check_existing_shards()

if not pending_shards:
    print("All shards already processed!")
else:
    print(f"Processing {len(pending_shards)} pending shards...")

Existing shards: 99/100
Pending shards: 1
Processing 1 pending shards...


In [12]:
# Process only pending shards

if pending_shards:
    max_workers = 6  # Conservative number of workers
    timeout_seconds = 2400  # 40 minutes timeout
    failed_shards = []
    
    print(f"Processing {len(pending_shards)} pending shards...")
    
    with ProcessPool(max_workers=max_workers) as pool:
        future = pool.map(process_shard, pending_shards, timeout=timeout_seconds)
        
        try:
            for i, result in enumerate(future.result()):
                if isinstance(result, Exception):
                    print(f"Shard {pending_shards[i]} failed: {result}")
                    failed_shards.append(pending_shards[i])
                else:
                    if i % 5 == 0:  # Progress indicator every 5 shards
                        print(f"Progress: {i+1}/{len(pending_shards)} shards completed")
        except Exception as e:
            print(f"Processing error: {e}")
    
    print(f"Processing completed. Failed shards: {len(failed_shards)}")
else:
    print("No pending shards to process!")

Processing 1 pending shards...
Processing shard 0 ...
Shard 0: 6657 examples to process


Saving the dataset (1/1 shards): 100%|██████████| 6657/6657 [00:00<00:00, 11291.57 examples/s]


Shard 0 completed in 1984.56 seconds
Progress: 1/1 shards completed
Processing completed. Failed shards: 0


### Collect all shards to form the processed dataset

In [13]:
from datasets import load_from_disk, concatenate_datasets

output = [dI for dI in os.listdir(root_directory) if os.path.isdir(os.path.join(root_directory,dI))]
datasets = []
for o in output:
    directory = root_directory + "/" + o
    try:
        shard = load_from_disk(directory)
        datasets.append(shard)
        print("%s loaded" % o)
    except:
        continue

shard_0 loaded
shard_1 loaded
shard_10 loaded
shard_11 loaded
shard_12 loaded
shard_13 loaded
shard_14 loaded
shard_15 loaded
shard_16 loaded
shard_17 loaded
shard_18 loaded
shard_19 loaded
shard_2 loaded
shard_20 loaded
shard_21 loaded
shard_22 loaded
shard_23 loaded
shard_24 loaded
shard_25 loaded
shard_26 loaded
shard_27 loaded
shard_28 loaded
shard_29 loaded
shard_3 loaded
shard_30 loaded
shard_31 loaded
shard_32 loaded
shard_33 loaded
shard_34 loaded
shard_35 loaded
shard_36 loaded
shard_37 loaded
shard_38 loaded
shard_39 loaded
shard_4 loaded
shard_40 loaded
shard_41 loaded
shard_42 loaded
shard_43 loaded
shard_44 loaded
shard_45 loaded
shard_46 loaded
shard_47 loaded
shard_48 loaded
shard_49 loaded
shard_5 loaded
shard_50 loaded
shard_51 loaded
shard_52 loaded
shard_53 loaded
shard_54 loaded
shard_55 loaded
shard_56 loaded
shard_57 loaded
shard_58 loaded
shard_59 loaded
shard_6 loaded
shard_60 loaded
shard_61 loaded
shard_62 loaded
shard_63 loaded
shard_64 loaded
shard_65 loaded

In [14]:
dataset = concatenate_datasets(datasets)
dataset.save_to_disk(config['data_folder'])
print('Dataset saved to %s' % config['data_folder'])

Saving the dataset (7/7 shards): 100%|██████████| 665622/665622 [00:29<00:00, 22564.21 examples/s]

Dataset saved to wikipedia_20231101.id.processed


In [15]:
# check the dataset size
dataset

Dataset({
    features: ['id', 'url', 'title', 'input_ids', 'phonemes'],
    num_rows: 665622
})

### Remove unneccessary tokens from the pre-trained tokenizer
The pre-trained tokenizer contains a lot of tokens that are not used in our dataset, so we need to remove these tokens. We also want to predict the word in lower cases because cases do not matter that much for TTS. Pruning the tokenizer is much faster than training a new tokenizer from scratch. 

In [16]:
from simple_loader import FilePathDataset, build_dataloader

file_data = FilePathDataset(dataset)
loader = build_dataloader(file_data, num_workers=32, batch_size=128)

In [17]:
special_token = config['dataset_params']['word_separator']

In [18]:
# get all unique tokens in the entire dataset

from tqdm import tqdm

unique_index = [special_token]
for _, batch in enumerate(tqdm(loader)):
    unique_index.extend(batch)
    unique_index = list(set(unique_index))

100%|██████████| 5200/5200 [00:50<00:00, 102.45it/s]


In [19]:
# get each token's lower case

lower_tokens = []
for t in tqdm(unique_index):
    word = tokenizer.decode([t])
    if word.lower() != word:
        t = tokenizer.encode([word.lower()])[0]
        lower_tokens.append(t)
    else:
        lower_tokens.append(t)

100%|██████████| 188335/188335 [00:02<00:00, 69927.92it/s]


In [20]:
lower_tokens = (list(set(lower_tokens)))

In [21]:
# redo the mapping for lower number of tokens

token_maps = {}
for t in tqdm(unique_index):
    word = tokenizer.decode([t])
    word = word.lower()
    new_t = tokenizer.encode([word.lower()])[0]
    token_maps[t] = {'word': word, 'token': lower_tokens.index(new_t)}

100%|██████████| 188335/188335 [00:19<00:00, 9614.14it/s] 


In [22]:
import pickle
with open(config['dataset_params']['token_maps'], 'wb') as handle:
    pickle.dump(token_maps, handle)
print('Token mapper saved to %s' % config['dataset_params']['token_maps'])

Token mapper saved to token_maps.pkl


### Test the dataset with dataloader


In [23]:
from dataloader import build_dataloader

train_loader = build_dataloader(dataset, batch_size=32, num_workers=0, dataset_config=config['dataset_params'])

177


In [24]:
_, (words, labels, phonemes, input_lengths, masked_indices) = next(enumerate(train_loader))

In [27]:
# Inspect shard content - lihat isi phoneme dari shard tertentu

from datasets import load_from_disk

def inspect_shard(shard_number, num_samples=5):
    """
    Melihat isi phoneme dari shard tertentu
    
    Args:
        shard_number: nomor shard yang ingin dilihat (0-99)
        num_samples: jumlah contoh yang ingin ditampilkan
    """
    shard_directory = f"{root_directory}/shard_{shard_number}"
    
    if not os.path.exists(shard_directory):
        print(f"Shard {shard_number} tidak ada di {shard_directory}")
        return
    
    try:
        # Load shard
        shard_data = load_from_disk(shard_directory)
        print(f"Shard {shard_number} info:")
        print(f"- Total samples: {len(shard_data)}")
        print(f"- Columns: {shard_data.column_names}")
        print("="*60)
        
        # Tampilkan beberapa contoh
        for i in range(min(num_samples, len(shard_data))):
            sample = shard_data[i]
            print(f"\nSample {i+1}:")
            print(f"Input IDs: {sample['input_ids'][:10]}{'...' if len(sample['input_ids']) > 10 else ''}")
            print(f"Phonemes: {sample['phonemes'][:10]}{'...' if len(sample['phonemes']) > 10 else ''}")
            
            # Decode beberapa token untuk melihat isi sebenarnya
            if len(sample['input_ids']) > 0:
                decoded_words = [tokenizer.decode([token_id]) for token_id in sample['input_ids'][:5]]
                print(f"Decoded words (first 5): {decoded_words}")
            
            print(f"Phonemes (first 5): {sample['phonemes'][:5]}")
            print("-"*40)
            
    except Exception as e:
        print(f"Error loading shard {shard_number}: {e}")

# Contoh penggunaan: lihat isi shard 0
inspect_shard(0, num_samples=3)

Shard 0 info:
- Total samples: 6657
- Columns: ['id', 'url', 'title', 'input_ids', 'phonemes']

Sample 1:
Input IDs: [24, 24, 2, 24, 24, 82734, 24, 3159, 22, 24]...
Phonemes: ['ˈasam', 'dˌɛoksˌiribˌonuklˈɛat', ',', 'lˈɛbih', 'dˈikənal', 'dˈɛŋan', 'siŋkˈatan', 'dnˈa', '(', 'bahˈasa']...
Decoded words (first 5): ['<unk>', '<unk>', ',', '<unk>', '<unk>']
Phonemes (first 5): ['ˈasam', 'dˌɛoksˌiribˌonuklˈɛat', ',', 'lˈɛbih', 'dˈikənal']
----------------------------------------

Sample 2:
Input IDs: [5305, 35337, 8445, 1260, 40509, 39, 21, 24, 24, 24]...
Phonemes: ['muhˈammad', 'ˈanwar', 'ˈɛl', '-', 'sˈadat', ';', ')', 'adˈalah', 'səˈɔraŋ', 'pˌolitˈikus']...
Decoded words (first 5): ['Muhammad', 'Anwar', 'el', '-', 'Sadat']
Phonemes (first 5): ['muhˈammad', 'ˈanwar', 'ˈɛl', '-', 'sˈadat']
----------------------------------------

Sample 3:
Input IDs: [56142, 27291, 24, 24, 90205, 5021, 24, 47071, 24, 24]...
Phonemes: ['dˈatuk', 'ˈazhar', 'mˈansɔr', 'adˈalah', 'ˈɔraŋ', 'mˌalajsˈia', 'pərtˈama